In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import train_test_split
import numpy as np
import optuna
from sklearn.model_selection import cross_validate

In [ ]:
training_data = np.genfromtxt('dataset.csv', delimiter=',', dtype=np.int32)

In [ ]:
X = training_data[:, :-1]
y = training_data[:, -1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=101)

In [ ]:
class Objective:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __call__(self, trial):
        
        params = {
            'solver': trial.suggest_categorical('solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']),
            'C': trial.suggest_loguniform('C', 0.0001, 10),
            'max_iter': trial.suggest_int('max_iter', 100, 100000)
        }
        
        model = LogisticRegression(**params)
        
        scores = cross_validate(model, X=self.X, y=self.y, scoring='accuracy', n_jobs=-1)
        
        return scores['test_score'].mean()
    

In [ ]:
objective = Objective(X_train, y_train)
study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=60)

print('params:', study.best_params)

In [ ]:
model = LogisticRegression(solver=study.best_params['solver'], C=study.best_params['C'], max_iter=study.best_params['max_iter'])

model.fit(X_train, y_train)
pred = model.predict(X_test)

print('正解率: {:.5f}%'.format(100 * accuracy_score(y_test, pred)))
print(confusion_matrix(y_test, pred))

In [ ]:
print('適合率: {:.5f}%'.format(100 * precision_score(y_test, pred)))
print('再現率: {:.5f}%'.format(100 * recall_score(y_test, pred)))